# 字符串
可以存储以下三种类型
* 字节串: byte string
* 整数: 取值范围与系统的长整数范围相同
* 浮点数: 取值范围与双精度浮点数相同

In [1]:
import time
import redis
conn = redis.Redis()

## 自增命令和自减命令

In [2]:
conn.incr('couter')

1

In [3]:
conn.decr('couter')

0

In [4]:
# python 客户端将 INCR 和 INCRBY 统一在了 incr 中，
conn.decr('couter', 10)

-10

In [5]:
conn.incr('couter', 10)

0

In [6]:
# 返回的仍然是一个字节流
conn.get('couter')

b'0'

In [7]:
# 因此在赋值的时候可以用字符串
conn.set('counter', '13')

True

In [8]:
# 即使用整数，在内部应该也会转
conn.set('counter', 10)

True

In [9]:
conn.get('counter')

b'10'

In [10]:
conn.set('counter', '12')

True

In [11]:
conn.incr('counter')

13

In [12]:
# 想要增加一个浮点数，还是需要用到 byfloat
conn.incrbyfloat('counter', 1.2)

14.2

## 对字节串一部分内容进行读取或者写入

In [13]:
conn.set('str', 'a')

True

In [14]:
conn.append('str','b')

2

In [15]:
# 可以追加给一个并不存在的key
conn.append('new_str', 'thisisanewstring')

32

In [16]:
conn.getrange('str', 0, 1)

b'ab'

In [17]:
# 将某一个偏移量的字节替换
conn.setrange('str', 1, 'd')

2

In [18]:
# 还可以用来在固定位置增加字符串长度
conn.setrange('str', 0, 'a word')

6

In [19]:
conn.get('str')

b'a word'

In [20]:
# 将字节看做是二进制，获得位串中偏移量为offset的二进制位值
conn.getbit('str', 30)

1

In [21]:
# 会返回二进制位串被设置之前的值
conn.setbit('str', 2, 0)

1

In [22]:
conn.get('str')

b'A word'

In [23]:
# 二进制位串中1的个数
conn.bitcount('str')

22

In [24]:
# 将多个二进制运算进行按位运算，保存在dest中
conn.bitop('and', 'str_', 'str')

6

In [25]:
conn.get('str_')

b'A word'

# 列表

## 常用的列表命令

In [26]:
# 生成列表不能是 lset，lset的功能是对已有的列表的某一个位置的值重新赋值
conn.rpush('new_list', 'some')

4

In [27]:
conn.rpush('new_list', 'last')

5

In [28]:
conn.lpush('new_list', 'first')

6

In [29]:
conn.rpush('new_list','new_last')

7

In [30]:
conn.lrange('new_list', 0, -1)

[b'first', b'a', b'b', b'c', b'some', b'last', b'new_last']

In [31]:
conn.lpop('new_list')

b'first'

In [32]:
conn.lpop('new_list')

b'a'

In [33]:
conn.lrange('new_list', 0, -1)

[b'b', b'c', b'some', b'last', b'new_last']

In [34]:
# 可以多个同时推入
conn.rpush('new_list', 'a', 'b', 'c')

8

In [35]:
conn.lrange('new_list', 0, -1)

[b'b', b'c', b'some', b'last', b'new_last', b'a', b'b', b'c']

In [36]:
# 修剪
conn.ltrim('new_list', 2, -1)

True

In [37]:
conn.lrange('new_list', 0, -1)

[b'some', b'last', b'new_last', b'a', b'b', b'c']

## 阻塞式列表弹出命令+列表之间移动元素命令
主要用在消息队列和任务队列实现的时候

In [38]:
# 这些都涉及到多进程交互，比较难在单进程中看到实际的功能
conn.rpush('list', 'item1', 'item2', 'item3')
conn.rpush('list2', 'item4')
# 阻塞式右pop一个list，得到该值，并将该值lpush进另一个list
conn.brpoplpush('list2', 'list', 1)

b'item4'

In [39]:
start = time.time()
result = conn.brpoplpush('list2', 'list', 5) is None
end = time.time()
print(int(end - start))
print(result)

5
True


In [40]:
conn.lrange('list', 0, -1)

[b'item4',
 b'item4',
 b'item1',
 b'item2',
 b'item3',
 b'item1',
 b'item2',
 b'item3']

类似其他的命令还有：BLPOP BRPOP RPOPLPUSH

# 集合

## 常用集合命令

In [41]:
# 将一个或多个元素放进集合里，并返回被添加元素当中原本并不存在于集合里面的元素数量
conn.sadd('set-key', 'a', 'b', 'c')

2

In [42]:
# 将一个或多个元素从集合里移除，并返回被移除元素的数量
conn.srem('set-key', 'c', 'd')

1

In [43]:
conn.srem('set-key', 'c', 'd')

0

In [44]:
# 检查是否存在于集合中
conn.sismember('set-key', 'a')

True

In [45]:
# 集合包含元素的数量
conn.scard('set-key')

4

In [46]:
# 返回集合包含的所有元素
conn.smembers('set-key')

{b'a', b'b', b'item2', b'item3'}

In [47]:
# 从集合里随机返回一个或多个元素，count为正时不允许重复，为负时可以重复
conn.srandmember('set-key', 4)

[b'b', b'item3', b'item2', b'a']

In [48]:
conn.srandmember('set-key', -8)

[b'item3', b'a', b'b', b'item2', b'item2', b'item3', b'a', b'a']

In [49]:
# 如果集合包含item，将item从一边移动到另一边
conn.smove('set-key', 'set-key2', 'a')

True

In [50]:
conn.smembers('set-key2')

{b'a'}

## 组合或处理多个集合

In [51]:
conn.sadd('skey1', 'a', 'b', 'c', 'd')
conn.sadd('skey2', 'c', 'd', 'e', 'f')

0

In [52]:
# 差集运算
conn.sdiff('skey1', 'skey2')

{b'a', b'b'}

In [53]:
# 差集运算并保存
conn.sdiffstore('skeydiff','skey1', 'skey2')
conn.smembers('skeydiff')

{b'a', b'b'}

In [54]:
# 交集
conn.sinter('skey1', 'skey2')

{b'c', b'd'}

In [55]:
# 并集
conn.sunion('skey1', 'skey2')

{b'a', b'b', b'c', b'd', b'e', b'f'}

# 散列

## 常用的散列操作

In [56]:
# 为散列里面的一个或多个键设置值
conn.hmset('hash-key', {'k1': 'v1', 'k2': 'v2', 'k3': 'v3'})

True

In [57]:
# 获取一个或多个值  HashMulti ， 单参数版本有HGET 和 HSET
conn.hmget('hash-key', ['k1', 'k2', 'k3'])

[b'v1', b'v2', b'v3']

In [58]:
# 散列长度
conn.hlen('hash-key')

5

In [59]:
# 删除散列里面一个或多个键值对，返回被删除的数量
conn.hdel('hash-key', 'k1', 'k4')

2

## 更高级的特性

In [60]:
# 是否存在
conn.hexists('hash-key', 'k3')

True

In [61]:
# 获取所有键
conn.hkeys('hash-key')

[b'sub-key1', b'k2', b'k3']

In [62]:
# 获取所有值
conn.hvals('hash-key')

[b'value1', b'v2', b'v3']

In [63]:
# 获取所有键值对
conn.hgetall('hash-key')

{b'k2': b'v2', b'k3': b'v3', b'sub-key1': b'value1'}

In [64]:
# 在键key存储的值上加上整数 increment（同样也有float的）
conn.hincrby('hash-key', 'k4')

1

In [65]:
conn.hexists('hash-key', 'k4')

True

# 有序集合
提供了分值处理方面的命令

## 基本操作命令

In [66]:
# 将带有给定分值的成员添加到有序集合 注意和散列创建的模式不一样
# 标准的先输入分值再输入键， python中不是这样
conn.zadd('zset-key', 'a', 3, 'b', 2, 'c', 1)

1

In [67]:
# 包含的成员数量
conn.zcard('zset-key')

3

In [68]:
# 将member成员的分值加上 increment
conn.zincrby('zset-key', 'c', 3)

4.0

In [69]:
# 返回member的分值
conn.zscore('zset-key', 'c')

4.0

In [70]:
# 返回排名, 0 是最小的
conn.zrank('zset-key', 'b')

0

In [71]:
# 返回介于min 和 max之间的成员数量
conn.zcount('zset-key', 0, 3)

2

In [72]:
# 移除给定成员     
conn.zrem('zset-key', 'b')

1

In [73]:
# 给出排名介于start 和 stop 中间的成员信息
conn.zrange('zset-key', 0, -1, withscores=True)

[(b'a', 3.0), (b'c', 4.0)]

## 其他高效操作

In [74]:
conn.zadd('zset-1', 'a', 1, 'b', 2, 'c', 3)
conn.zadd('zset-2', 'b', 4, 'c', 1, 'd', 0)

0

In [75]:
# 交集运算 + 权重相加
conn.zinterstore('zset-i', ['zset-1', 'zset-2'])

2

In [76]:
conn.zrange('zset-i', 0, -1, withscores=True)

[(b'c', 4.0), (b'b', 6.0)]

In [77]:
conn.zinterstore('zset-i-weight',{'zset-1': 0.5, 'zset-2': 0.5})

2

In [78]:
conn.zrange('zset-i-weight', 0, -1, withscores=True)

[(b'c', 2.0), (b'b', 3.0)]

In [79]:
# 并集运算
conn.zunionstore('zset-u-weight', {'zset-1': 0.4, 'zset-2': 0.6})
conn.zrange('zset-u-weight', 0, -1, withscores=True)

[(b'd', 0.0), (b'a', 0.4), (b'c', 1.8000000000000003), (b'b', 3.2)]

In [80]:
# range 从大到小排列
conn.zrevrange('zset-u-weight', 0, -1, withscores=True)

[(b'b', 3.2), (b'c', 1.8000000000000003), (b'a', 0.4), (b'd', 0.0)]

In [81]:
# rangebyscore
# rabgebyscore 从大到小排
conn.zrangebyscore('zset-u-weight', 0, 3)

[b'd', b'a', b'c']

In [82]:
# 按照rangebyscore 删除
# 按照rangebyrank 删除
conn.zremrangebyrank('zset-u-weight', 0, 2)

3

In [83]:
conn.zrange('zset-u-weight', 0, -1, withscores=True)

[(b'b', 3.2)]

# 发布与订阅

为方便演示，使用一个线程来执行

In [84]:
import threading
def publisher(n):
    time.sleep(1)
    for i in range(n):
        conn.publish('channel', i)
        time.sleep(1)
        
def run_pubsub():
    threading.Thread(target=publisher, args=(3,)).start()
    # 创建发布与订阅的对象
    pubsub = conn.pubsub()
    pubsub.subscribe(['channel'])
    count = 0
    for item in pubsub.listen():
        print(item)
        count += 1
        if count == 3:
            pubsub.unsubscribe()
        if count == 4:
            break

In [85]:
run_pubsub()

{'type': 'subscribe', 'pattern': None, 'channel': b'channel', 'data': 1}
{'type': 'message', 'pattern': None, 'channel': b'channel', 'data': b'0'}
{'type': 'message', 'pattern': None, 'channel': b'channel', 'data': b'1'}
{'type': 'unsubscribe', 'pattern': None, 'channel': b'channel', 'data': 0}


但是说到底作为一个内存数据库，如果读取消息不够快造成积压的话，还是会有问题的  
此外并没有保证等幂的措施，如果在传输某个数据失败了，没有办法知道失败并重新传输  

# 其他命令

## 排序 SORT
功能强大到有点复杂，可以根据字符串，列表，集合，有序集合，散列这五种键里面存储的数据，对列表、集合、有序集合进行排序，类似 order by

In [86]:
conn.rpush('sort-input', 23, 15, 110, 7)

8

In [87]:
conn.sort('sort-input')

[b'7', b'7', b'15', b'15', b'23', b'23', b'110', b'110']

In [88]:
conn.sort('sort-input', desc=True)

[b'110', b'110', b'23', b'23', b'15', b'15', b'7', b'7']

In [89]:
# 以字符串的形式来比较大小
conn.sort('sort-input', alpha=True)

[b'110', b'110', b'15', b'15', b'23', b'23', b'7', b'7']

In [90]:
# orderby 某些散列的信息对列表排序，有点根据散列中某个值，对散列的id们进行排序的意思
conn.hset('d-7', 'field', 5)
conn.hset('d-15', 'field', 1)
conn.hset('d-23', 'field', 9)
conn.hset('d-110', 'field', 3)
conn.sort('sort-input', by='d-*->field')

[b'15', b'15', b'110', b'110', b'7', b'7', b'23', b'23']

In [91]:
# get 还可以转成想要的相关的其他数据
conn.sort('sort-input', by='d-*->field', get='d-*->field')

[b'1', b'1', b'3', b'3', b'5', b'5', b'9', b'9']

## 基本的redis事务（部分）
可以保证一个客户端在不被其他客户端打断的情况下执行多个命令，相当于一个打包程序   
在客户端使用的时候，需要先输入MULTI，再输入所有想做的操作，最后用EXEC做结尾 

In [92]:
# 先展示一下在没有事务的时候可能会引发的问题
def notrans():
    print(conn.incr('notrans:'))
    time.sleep(2)
    conn.incr('notrans:', -1)

for i in range(3):
    threading.Thread(target=notrans).start()
    time.sleep(2)

1
2
1


In [93]:
# 使用事务来处理问题
# 这里由于时间大小关系，可能会导致某两个线程在同一时间抢占print管道，只能输出一个
def trans():
    pipeline = conn.pipeline()
    pipeline.incr('trans:')
    # 这个sleep其实不会放在pipeline中，但是会对代码造成影响
    time.sleep(2)
    conn.incr('trans:', -1)
    res = pipeline.execute()
    print(res[0])

for i in range(3):
    threading.Thread(target=trans).start()
    time.sleep(2)

0
0
0


In [94]:
conn.delete('trans:')

1

In [95]:
pipeline = conn.pipeline()
pipeline.incr('trans:')
# 这个sleep其实不会放在pipeline中
time.sleep(.5)
conn.incr('trans:', -1)
res = pipeline.execute()
print(res[0])

0


## 键的过期时间

In [96]:
conn.set('key', 'value')

True

In [97]:
conn.set('key', 'value')
conn.expire('key', 2)
time.sleep(2)
conn.get('key') is None

True

In [98]:
conn.setex('key', 'value2', 100)

True

In [99]:
# 查看给定键距离过期还有多少秒
conn.ttl('key')

100

In [100]:
# 移除键的过期时间
conn.persist('key')

True

In [101]:
conn.ttl('key') is None

True